In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
# Load in our data
big_df = pd.read_csv("../data/big_df.csv")
big_df = big_df.drop(columns="lev1")
big_df

,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,Orthodox,Orthodox,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.416667,99.571429,0.116189

In [3]:
# removing fights that have not taken place yet
df = big_df[big_df["winner"] != "tbd"]
df

,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
14,29.0,38.0,72,73,Darren Till,Derek Brunson,74,77,Southpaw,Southpaw,283,1015,614,1704,265,527,593,1126,5,29,9,93,0,4,571,3420,4,5,0,0,50,118,67,171,186,355,521,794,97,73,117,97,207,341,548,766,49,83,62,115,77,122,95,181,444,729,960,1240,329,421,834,913,5,1,37,16,2,0,885,194,3,5,0,0,50,118,67,171,186,355,521,794,97,73,117,97,207,341,548,766,49,83,62,115,77,122,95,181,7401,8905,4,5,0,1,2,7,1,1,1,0,1,4,6,13,3,5,1,0,0.666667,0.722222,6.838855,3.550814,0.595657,0.468028,2.391915,0.795059,0.491859,2.297586,0.559236,0.822010,4.911847,2.836609,2.391915,0.795059,0.491859,2.297586,0.559236,0.822010,1.927007,0.714206,23.043234,1.307131,21.736103,0.311828,99.9375,0.195396,0.006738,0.948767,3,1,2.294285,2.148358,0.460912,0.446880,1.507904,0.405351,0.786380,1.678152,0.397244,0.624240,3.599514,2.667207,1.507904,0.405351,0.786380,1.678152,0.397244,0.624240,-1.305229,-0.518849,4.629104,7.174706,-2.545602,0.555556,99.864865,0.040535,0.04

In [4]:
# Label encoder for stance and weight class
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
df['red_stance'] = number.fit_transform(df['red_stance'].astype('str'))
df['blue_stance'] = number.fit_transform(df['blue_stance'].astype('str'))
df['weight_class'] = number.fit_transform(df['weight_class'].astype('str'))
df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
14,29.0,38.0,72,73,Darren Till,Derek Brunson,74,77,1,1,283,1015,614,1704,265,527,593,1126,5,29,9,93,0,4,571,3420,4,5,0,0,50,118,67,171,186,355,521,794,97,73,117,97,207,341,548,766,49,83,62,115,77,122,95,181,444,729,960,1240,329,421,834,913,5,1,37,16,2,0,885,194,3,5,0,0,50,118,67,171,186,355,521,794,97,73,117,97,207,341,548,766,49,83,62,115,77,122,95,181,7401,8905,4,5,0,1,2,7,1,1,1,0,1,4,6,13,3,5,1,0,0.666667,0.722222,6.838855,3.550814,0.595657,0.468028,2.391915,0.795059,0.491859,2.297586,0.559236,0.822010,4.911847,2.836609,2.391915,0.795059,0.491859,2.297586,0.559236,0.822010,1.927007,0.714206,23.043234,1.307131,21.736103,0.311828,99.9375,0.195396,0.006738,0.948767,3,1,2.294285,2.148358,0.460912,0.446880,1.507904,0.405351,0.786380,1.678152,0.397244,0.624240,3.599514,2.667207,1.507904,0.405351,0.786380,1.678152,0.397244,0.624240,-1.305229,-0.518849,4.629104,7.174706,-2.545602,0.555556,99.864865,0.040535,0.040535,1.509434,

In [5]:
# setting our X and y values
X = df.drop(columns=["winner", "red_name", "blue_name", "date", "event"])
y = df["winner"]
feature_names = X.columns

In [6]:
# illustrate the concept of resampling with a logistic regression model before applying it to our random forest
clf_0 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_0 = clf_0.predict(X)
pred_y_0

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['red', 'red', 'red', ..., 'red', 'red', 'red'], dtype=object)

In [7]:
# How's the accuracy?
print(accuracy_score(pred_y_0, y))

0.6670656634746922


In [8]:
print( np.unique( pred_y_0 ) )

['blue' 'red']


In [9]:
df["winner"].value_counts()

red     3830
blue    2018
Name: winner, dtype: int64

In [10]:
# Separate majority and minority classes
df_majority = df[df.winner=="red"]
df_minority = df[df.winner=="blue"]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority), # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.winner.value_counts()

red     3830
blue    3830
Name: winner, dtype: int64

In [11]:
df_upsampled

,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
14,29.0,38.0,72,73,Darren Till,Derek Brunson,74,77,1,1,283,1015,614,1704,265,527,593,1126,5,29,9,93,0,4,571,3420,4,5,0,0,50,118,67,171,186,355,521,794,97,73,117,97,207,341,548,766,49,83,62,115,77,122,95,181,444,729,960,1240,329,421,834,913,5,1,37,16,2,0,885,194,3,5,0,0,50,118,67,171,186,355,521,794,97,73,117,97,207,341,548,766,49,83,62,115,77,122,95,181,7401,8905,4,5,0,1,2,7,1,1,1,0,1,4,6,13,3,5,1,0,0.666667,0.722222,6.838855,3.550814,0.595657,0.468028,2.391915,0.795059,0.491859,2.297586,0.559236,0.822010,4.911847,2.836609,2.391915,0.795059,0.491859,2.297586,0.559236,0.822010,1.927007,0.714206,23.043234,1.307131,21.736103,0.311828,99.937500,0.195396,0.006738,0.948767,3,1,2.294285,2.148358,0.460912,0.446880,1.507904,0.405351,0.786380,1.678152,0.397244,0.624240,3.599514,2.667207,1.507904,0.405351,0.786380,1.678152,0.397244,0.624240,-1.305229,-0.518849,4.629104,7.174706,-2.545602,0.555556,99.864865,0.040535,0.040535,1.50943

In [12]:
# Separate input features (X) and target variable (y)
y = df_upsampled.winner
X = df_upsampled.drop(columns=["winner", "red_name", "blue_name", "date", "event"], axis=1)
 
# Train model
clf_1 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_1 = clf_1.predict(X)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_1 ) )

['blue' 'red']


C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
# How's our accuracy?
print(accuracy_score(y, pred_y_1))

0.6140992167101827


In [14]:
# Now that we can see resampling will help us increase accuracy on a logistic regression, lets see on random forest
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
# our decision tree score has increased dramatically
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7592689295039164

In [16]:
sorted(zip(clf.feature_importances_, X), reverse=True)

[(0.03910927401460604, 'red_age'),
 (0.03657396147585164, 'red_SigStkAbsPerMin'),
 (0.027534973967223656, 'blue_age'),
 (0.020560695540787658, 'red_CtrlDiff'),
 (0.01777936928038324, 'blue_sumTDsSeen'),
 (0.01777178615030516, 'red_StkGndAbsPerMin'),
 (0.016651853879993475, 'red_TotStkAcc'),
 (0.0161175188802983, 'blue_SigStkPerMinDiff'),
 (0.01425412945735028, 'blue_heightIN'),
 (0.014138381713250946, 'red_reach'),
 (0.014126853452849426, 'red_SigStkPerMinDiff'),
 (0.01332917112006413, 'red_heightIN'),
 (0.013022092142588261, 'red_StkLegPerMin'),
 (0.012952221374876184, 'weight_class'),
 (0.012794395524432775, 'blue_TotStkPerMinDiff'),
 (0.012761280229817509, 'red_sumCtrldSec'),
 (0.012732978857647114, 'blue_SigStkAcc'),
 (0.012538931460756329, 'red_winPCT'),
 (0.012348173229869568, 'blue_reach'),
 (0.011610945227025446, 'red_sumKedD'),
 (0.01151951989216596, 'red_heightDiff'),
 (0.01147356422365311, 'blue_CtrlSecPerMin'),
 (0.010974866200938412, 'red_StkHeadPerMin'),
 (0.0109509604734

In [17]:
# and so has our random forest
rf = RandomForestClassifier(n_estimators=105)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8318537859007833

In [18]:
preds = rf.predict(X)

In [19]:
filename = "../saved_models/rf_up_model.sav"
pickle.dump(rf, open(filename, 'wb'))

In [20]:
# Repeating proccess by down sampling red wins

In [21]:
# Separate majority and minority classes
df_majority = df[df.winner=="red"]
df_minority = df[df.winner=="blue"]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority), # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.winner.value_counts()

blue    2018
red     2018
Name: winner, dtype: int64

In [22]:
# illustrating this concept with a logistic regression model before applying it to our random forest

In [23]:
# Separate input features (X) and target variable (y)
y = df_downsampled.winner
X = df_downsampled.drop(columns=["winner", "red_name", "blue_name", "event", "date"], axis=1)
 
# Train model
clf_2 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_2 = clf_2.predict(X)
 
# Is our model still predicting just one class?
pred_y_2

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['red', 'red', 'red', ..., 'red', 'red', 'red'], dtype=object)

In [24]:
# How's our accuracy?
print( accuracy_score(y, pred_y_2) )

0.6157086223984143


In [25]:
# repeating on down-sampled data

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [27]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5510406342913776

In [28]:
sorted(zip(clf.feature_importances_, X), reverse=True)

[(0.045303171271659864, 'red_SigStkAbsPerMin'),
 (0.040685424260675514, 'red_age'),
 (0.02578037130941173, 'blue_age'),
 (0.02054966220817174, 'red_reach'),
 (0.01998270589453543, 'red_sumCtrldSec'),
 (0.018489785184911064, 'red_StkGndPerMin'),
 (0.017964789339891523, 'blue_reachDiff'),
 (0.017529715889998028, 'blue_TotStkAcc'),
 (0.017482465182534285, 'blue_CtrldSecPerMin'),
 (0.017363673268886014, 'red_TotStkPerMinDiff'),
 (0.016361815220486423, 'blue_SigStkPerMinDiff'),
 (0.016208415777175123, 'blue_reach'),
 (0.01616714491471226, 'red_SigStkAcc'),
 (0.015511983251213505, 'blue_TDAcc'),
 (0.01435967709924512, 'red_TotStkAbsPerMin'),
 (0.014021797685071023, 'weight_class'),
 (0.01393400761340486, 'blue_StkHeadPerMin'),
 (0.013775899103240135, 'red_sumReversed'),
 (0.01310395372564201, 'blue_TotStkPerMin'),
 (0.01194867818526651, 'blue_TotStkPerMinDiff'),
 (0.011929099760972006, 'red_TDsPerMin'),
 (0.011219782308150041, 'red_TDsAbsPerMin'),
 (0.010646876645837725, 'red_StkDistAbsPerMi

In [29]:
rf = RandomForestClassifier(n_estimators=105)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.5896927651139743

In [30]:
preds = rf.predict(X)

In [31]:
sorted(zip(rf.feature_importances_, X), reverse=True)

[(0.019570961063726685, 'red_age'),
 (0.01448066477645982, 'blue_age'),
 (0.013711127553877354, 'red_SigStkAbsPerMin'),
 (0.013473762838617934, 'red_reachDiff'),
 (0.012451027322262687, 'red_TotStkAbsPerMin'),
 (0.011441825210748616, 'blue_reach'),
 (0.01133338360993032, 'blue_reachDiff'),
 (0.011133922924842473, 'red_reach'),
 (0.010944990263784987, 'weight_class'),
 (0.010669669355426558, 'red_heightIN'),
 (0.010044068237288808, 'red_CtrlDiff'),
 (0.009863427168896865, 'blue_heightIN'),
 (0.009619251818545563, 'red_CtrlSecPerMin'),
 (0.009597588351731304, 'red_CtrldSecPerMin'),
 (0.009246175215995169, 'red_sumCtrldSec'),
 (0.009204806653340083, 'red_SigStkPerMinDiff'),
 (0.00900405234359591, 'red_TotStkPerMinDiff'),
 (0.008687209392501987, 'red_TotStkAcc'),
 (0.008433950342184368, 'blue_TotStkAbsPerMin'),
 (0.008383134470918856, 'red_heightDiff'),
 (0.008332948982213085, 'blue_heightDiff'),
 (0.008276141442607184, 'red_TDsPerMin'),
 (0.008258411831306642, 'red_StkDistPerMin'),
 (0.00

In [32]:
# saving this model too, even though it preforms worse than our upsampled model to compare on future observations
filename = "../saved_models/rf_down_model.sav"
pickle.dump(rf, open(filename, "wb"))